In [ ]:
# Mount The Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Import Libraries
import os
import numpy as np
import pandas as pd
import time 
import datetime
import pickle
pd.options.mode.chained_assignment = None
print("Libraries are imported")

Libraries are imported


In [ ]:
# Define Path of GPS Data
path = '---------------------------------'
os.chdir(path)

In [ ]:
# Read GPS Data
df_list = os.listdir()
df = []
for i in df_list:
    with open(f'{i}', 'rb') as f:
        df.append(f.readlines())

In [ ]:
temp = []
for i in range(len(df_list)):
    txt = df_list[i]
    User = ''.join((ch if ch in '0123456789' else '') for ch in txt)
    temp.append(User)

In [ ]:
# Define User list
User_list = []
for i in temp:
    User_list.append(int(i))

In [ ]:
# Read Latitude, Longitude, Altitude, Date, Time, User
Lat = []
Lon = []
Alt = []
Date = []
Time = []
User = []

for i in range(len(df)):
    del df[i][0:6]
    del df[i][-1]
    for j in range(len(df[i])):
        line = str(df[i][j]).split(",")
        if len(line) == 7:
            Lat.append(line[0][2:])
            Lon.append(line[1])
            Alt.append(line[4])
            Date.append(line[5])
            Time.append(line[6][0:8])
            User.append(User_list[i])  

In [ ]:
Data = {'Latitude':Lat, 'Longitude':Lon, 'Altitude':Alt, 'Date':Date, 'Time':Time, 'User':User}
DF = pd.DataFrame(Data)

In [ ]:
DF['TS1'] = DF['Date'] + " " + DF['Time']

In [ ]:
# Change Date and Time to Timestamp
ST = []
for i in DF['TS1']:
    element = datetime.datetime.strptime(i,"%Y-%m-%d %H:%M:%S") 
    ST.append(datetime.datetime.timestamp(element))

In [ ]:
DF['TS'] = ST
DF.drop('TS1', inplace = True, axis = 1)

In [ ]:
# Define Trip
DF['Trip'] = np.nan
DF['Trip'].iloc[0] = 1

In [ ]:
# If User Changes it is a New Trip
# If Time Difference between 2 GPS rows higher than 60 Seconds it is a New Trip
Limit = 600
counter = 2
for i in range(len(DF)-1):
  Time_Dif = DF['TS'].iloc[i+1] - DF['TS'].iloc[i]
  if DF['User'].iloc[i] != DF['User'].iloc[i+1] or Time_Dif > Limit:
    DF['Trip'].iloc[i+1] = counter
    counter += 1

In [ ]:
# Total Number of Trips
All_Trips = counter - 1
All_Trips

20202

In [ ]:
DF.fillna(method='ffill', inplace = True)

In [ ]:
DF.head()

,Latitude,Longitude,Altitude,Date,Time,User,TS,Trip
0,39.974408918,116.303522101,40753.5306944444,2011-07-29,12:44:12,100,1.311943e+09,1.0
1,39.974397078,116.303526932,40753.5307060185,2011-07-29,12:44:13,100,1.311943e+09,1.0
2,39.973982524,116.303621837,40753.5307291667,2011-07-29,12:44:15,100,1.311943e+09,1.0
3,39.973943291,116.303632641,40753.5307407407,2011-07-29,12:44:16,100,1.311943e+09,1.0
4,39.973937148,116.303639667,40753.5307523148,2011-07-29,12:44:17,100,1.311943e+09,1.0


In [ ]:
# Define Path for Pickle Data
path = '---------------------------------'
os.chdir(path)

In [ ]:
# Save All GPS Trips in a DataFrame
with open("001_All_GeoLife_GPS.pickle", 'wb') as f: 
    pickle.dump(DF, f)